In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
os.chdir("/Users/deeptimurthy/BENG203")
wd = '/Users/deeptimurthy/BENG203'

In [ ]:
import pandas as pd

cancer_df = pd.read_csv(wd + "/data/pnas_normal_tpm.txt", sep="\t", index_col=0)
normal_df = pd.read_csv(wd + "/data/pnas_tpm_96_nodup.txt", sep="\t", header=None)

cancer_df.columns = [f'C{i+1}' for i in range(cancer_df.shape[1])]
normal_df.set_index(normal_df.columns[0], inplace=True)
normal_df.columns = [f'N{i+1}' for i in range(normal_df.shape[1])]

combined_data = pd.concat([cancer_df, normal_df], axis=1, join='inner')
combined_data.head()

combined_data.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,N87,N88,N89,N90,N91,N92,N93,N94,N95,N96
ENSG00000000003,26.432148,17.260895,0.000000,69.795165,19.331552,0.000000,2.928341,53.467303,14.997727,25.816814,...,14.134044,10.803910,6.068982,12.714350,15.983392,0.000000,38.189128,15.608352,51.519267,30.805593
ENSG00000000005,24.817762,5.402219,7.557640,0.000000,0.000000,14.330953,0.000000,16.733900,14.081717,0.000000,...,0.000000,12.172853,136.759411,28.650721,0.000000,0.000000,5.976111,6.280734,241.863225,0.000000
ENSG00000000419,66.208114,0.000000,10.081027,0.000000,18.158365,9.557926,0.000000,0.000000,0.000000,0.000000,...,79.657691,8.118597,0.000000,0.000000,0.000000,179.178412,15.942897,0.000000,0.000000,0.000000
ENSG00000000457,8.127159,6.318156,15.910242,15.328639,19.105460,6.704296,34.729107,0.000000,2.195899,20.102628,...,18.624986,15.660412,33.988677,0.000000,36.858404,83.788404,18.172309,2.938248,14.547636,10.148436
ENSG00000000460,8.035515,20.406574,0.000000,17.681754,11.019179,1.933370,0.000000,18.060385,10.131976,23.188602,...,17.903438,19.706680,23.062537,1.932615,28.344382,3.020339,27.411791,1.694651,3.729079,7.804221


In [64]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')
mapping_df = dataset.query(attributes=['ensembl_gene_id', 'hgnc_symbol'])
mapping_df.columns = ['Gene stable ID', 'HGNC symbol']
mapping_df = mapping_df[mapping_df['HGNC symbol'] != '']

ensg_to_symbol = dict(zip(mapping_df['Gene stable ID'], mapping_df['HGNC symbol']))

combined_data['gene_symbol'] = combined_data.index.map(ensg_to_symbol)
combined_data = combined_data[combined_data['gene_symbol'].notnull()]
combined_data.set_index('gene_symbol', inplace=True)
combined_data.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,N87,N88,N89,N90,N91,N92,N93,N94,N95,N96
gene_symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,26.432148,17.260895,0.000000,69.795165,19.331552,0.000000,2.928341,53.467303,14.997727,25.816814,...,14.134044,10.803910,6.068982,12.714350,15.983392,0.000000,38.189128,15.608352,51.519267,30.805593
TNMD,24.817762,5.402219,7.557640,0.000000,0.000000,14.330953,0.000000,16.733900,14.081717,0.000000,...,0.000000,12.172853,136.759411,28.650721,0.000000,0.000000,5.976111,6.280734,241.863225,0.000000
DPM1,66.208114,0.000000,10.081027,0.000000,18.158365,9.557926,0.000000,0.000000,0.000000,0.000000,...,79.657691,8.118597,0.000000,0.000000,0.000000,179.178412,15.942897,0.000000,0.000000,0.000000
SCYL3,8.127159,6.318156,15.910242,15.328639,19.105460,6.704296,34.729107,0.000000,2.195899,20.102628,...,18.624986,15.660412,33.988677,0.000000,36.858404,83.788404,18.172309,2.938248,14.547636,10.148436
FIRRM,8.035515,20.406574,0.000000,17.681754,11.019179,1.933370,0.000000,18.060385,10.131976,23.188602,...,17.903438,19.706680,23.062537,1.932615,28.344382,3.020339,27.411791,1.694651,3.729079,7.804221


In [65]:
# PCA to get the top 100 genes
X = combined_data.T 

# PCA
pca = PCA()
pca.fit(X)

loadings = pca.components_.T  

# importance vals
k = 2
importance_scores = np.sum(loadings[:, :k]**2, axis=1)

# Get top 100 genes
n_genes = min(100, combined_data.shape[0])
top_100_genes = combined_data.index[np.argsort(importance_scores)[::-1][:n_genes]]

print("Top 100 genes:", top_100_genes.tolist())

Top 100 genes: ['RNY1', 'RNY3', 'MT-CYB', 'MT-TP', 'MT-ND3', 'MT-ND5', 'MT-RNR2', 'MT-ND4', 'MT-ND4L', 'MT-ND1', 'MT-ND2', 'MT-CO3', 'MT-ND6', 'MT-TV', 'MT-RNR1', 'MT-CO2', 'RNY4', 'MT-ATP6', 'MT-ATP8', 'HBB', 'MT-CO1', 'RNA5SP298', 'MT-TT', 'MT-TG', 'MT-TH', 'MT-TL1', 'MT-TF', 'MT-TL2', 'RNA5SP202', 'MT-TI', 'MT-TM', 'MT-TR', 'RNA5SP141', 'RNA5SP150', 'RN7SL2', 'MT-TW', 'RNA5SP145', 'MTATP6P1', 'MT-TS2', 'MT-TD', 'MTND1P23', 'RNU4-2', 'IGHD1-14', 'PPBP', 'RAC2', 'RNA5SP243', 'MTND2P28', 'RNA5SP387', 'IGHD4-11', 'IGHD5-24', 'MT-TC', 'RNA5SP338', 'MT-TY', 'RNA5SP263', 'MT-TE', 'TRBJ1-5', 'VTRNA1-1', 'RNA5SP355', 'RNU6-411P', 'RNA5SP119', 'MIR1911', 'MTND4P12', 'RNU4ATAC6P', 'TRBJ1-1', 'TMSB4X', 'IGHV1-24', 'MIR8078', 'RNU6-253P', 'RNU6-1031P', 'KRAS', 'MIR653', 'MIR1972-2', 'FTL', 'MIR449A', 'RNA5SP149', 'SNORD111', 'RNA5SP148', 'RNY3P1', 'RNU2-55P', 'MIR8052', 'RNA5SP339', 'MIR610', 'RNU6-172P', 'RNU7-24P', 'RNA5SP116', 'RNA5SP450', 'RNU6-771P', 'MIR515-1', 'RNU6-120P', 'RNA5SP99', 'MI